In [ ]:
!git clone https://github.com/LukaMelinc/NMRV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install necessary libraries
!pip install albumentations segmentation-models-pytorch ultralytics -q

In [ ]:
from ultralytics import YOLO
import os
import numpy as np

model = YOLO('yolov8m.pt')

model.train(
    amp=False,
    data="/content/NMRV/Seminar/data_RGB.yaml",           # File za dataset
    imgsz=640,                  # max. velikost ki jo zahtevamo
    epochs=3,                 # Št. epoch
    batch=4,                    # Velikost batcha
    lr0 = 0.001,                # Initial learning rate
    lrf=0.01,                   # Learning Rate Factor
    cos_lr = False,             # linearno ali kosinusno zmanjšanje
    patience=20,                # Early stop trigger
    device="cuda",              # Rabi GPU
    project="yolo_ladica",      # Directory za training loge
    name="OvireNaVodi",         # Sub-folder name
    workers=1,                  # Št. corov za loadat podatke
    show=True,                  # Prikaži rezultate
    save=True,                  # Shrani naučene uteži

    # Augmentacija
    fliplr = 0.5,
    translate=0.1,
    erasing=0.1,
    hsv_h=0.1,
    hsv_v=0.1,
    hsv_s=0.1,
    degrees=10
)



In [ ]:
# Test the model on the test dataset
test_results = model.val(
    data="/content/NMRV/Seminar/data_RGB.yaml",  # Path to the dataset configuration file
    imgsz=640,                  # Image size for evaluation
    device="cuda",              # Use GPU for evaluation
    split='test'                # Specify the test split
)

def calculate_f1_score(precision, recall):
    if precision + recall > 0:
        return 2 * (precision * recall) / (precision + recall)
    else:
        return 0.0

# Print the test results
print("Test Results:")
print(test_results)

# Extract precision and recall from results_dict
results_dict = test_results.results_dict
precision = results_dict.get('metrics/precision(B)', 0)
recall = results_dict.get('metrics/recall(B)', 0)

# Calculate F1 score
f1_score = calculate_f1_score(precision, recall)

# Print the F1 score
print(f"F1 Score: {f1_score:.4f}")
